## Init in-memory sql db

In [1]:
import json
import sqlite3
import os

def init_db():
    def load_jsonl(conn, table, path):
        with open(path) as f:
            for line in f:
                row = json.loads(line)
                cols = ", ".join(row.keys())
                vals = ", ".join(f":{k}" for k in row.keys())
                conn.execute(
                    f"INSERT INTO {table} ({cols}) VALUES ({vals})",
                    row
                )

    conn = sqlite3.connect(":memory:", check_same_thread=False)

    conn.executescript(open("../backend/db/schema.sql").read())

    load_jsonl(conn, "products", "../ai_services/data/products.jsonl")
    conn.commit()

    conn.row_factory = sqlite3.Row
    
    return conn

In [2]:
conn = init_db()

## Init tools

### Product search

In [3]:
from langchain.tools import tool

from backend.app.schemas.product import Product

@tool
def product_search(sql_query: str) -> list[Product]:
    """
    Search for products in the database using SQL.
    
    Database schema:
    - Table: products
    - Columns: id (INTEGER), name (TEXT), description (TEXT), price (REAL), stock_quantity (INTEGER), supplier (TEXT), review_stars (INTEGER 1-5)
    
    Examples:
    - SELECT * FROM products WHERE name LIKE '%bread%'
    - SELECT * FROM products WHERE price < 10 AND stock_quantity > 0
    - SELECT * FROM products WHERE name LIKE '%bacon%' OR name LIKE '%lettuce%' OR name LIKE '%tomato%'
    
    Always use LIKE with % wildcards for name searches. Quote string literals properly.
    """

    # Execute query
    cursor = conn.execute(sql_query)
    rows = cursor.fetchall()

    # Convert rows to Product objects
    products = [Product(**row) for row in rows]

    return products

In [4]:
product_search.invoke("SELECT * FROM products WHERE name LIKE '%bread%'")

[Product(id=1, name='Whole Wheat Bread Loaf', description='Sliced whole wheat sandwich bread, 24 oz loaf', price=3.49, stock_quantity=120, supplier='Golden Grain Foods', review_stars=4)]

### Compliance checker

In [5]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

from backend.app.schemas.compliance_assessment import ComplianceAssessment

with open('../ai_services/data/policy.txt') as f:
    policy_text = f.read()

compliance_model = init_chat_model(model="gpt-4.1-mini", temperature=0)

compliance_agent = create_agent(
    system_prompt=f"""
    # Role
    You are a compliance checker for Direct Supply, a B2B procurement company that sells products to senior living communities. Your job is to evaluate product orders against company policies and identify any compliance violations.

    # Policy
    {policy_text}

    # Task
    Given a product order and the user's original query, determine if it complies with the company policies. If there are any violations, identify the specific products and the reasons for non-compliance. Return a structured report indicating whether the order is compliant and detailing any violations.
    """,
    response_format=ComplianceAssessment,
    model=compliance_model
)

In [6]:
from backend.app.schemas.product_order import ProductOrder
from backend.app.schemas.compliance_assessment import ComplianceAssessment

@tool
def compliance_checker(product_order: ProductOrder, original_user_query: str) -> ComplianceAssessment:
    """
    Evaluate a product order for compliance with company policies.
    """
    inputs = {
        'messages': [
            {
                'role': 'user', 
                'content': f"""
                Original User Query: {original_user_query}
                Product Order: {product_order.model_dump_json()}

                Determine if the order complies with the company policies. If there are any violations, identify the specific products and the reasons for non-compliance. Return a structured report indicating whether the order is compliant and detailing any violations.
                """
            }
        ]
    }
    assessment = compliance_agent.invoke(inputs)

    return assessment

### Calculator

In [7]:
@tool
def calculator(expression: str) -> str:
    """
    A simple calculator tool that evaluates basic arithmetic expressions.
    Example inputs:
    - 2 + 2
    - 10 / 5
    - (3 * 4) - 5
    """
    try:
        # WARNING: eval can be dangerous in production code. This is just for demonstration.
        result = eval(expression, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"

## Init agent

In [8]:
from langchain.agents import create_agent
agent_model = init_chat_model(model="gpt-4.1-mini", temperature=0)
tools = [product_search, compliance_checker, calculator]
agent = create_agent(
    model=agent_model, 
    tools=tools,
    system_prompt="You are a procurement assistant that helps procurement specialists for Direct Supply, a B2B procurement company that sells products to senior living communities."
)

## Invoke agent loop

In [12]:
# Initial messages from the user; rerun this cell to reset the conversation
initial_msg = input('Enter user message: ') or 'I need to procure products for 30 blt sandwiches. Create a valid product order for me.'

messages = {
    'messages': [
        {
            'role': 'user',
            'content': initial_msg
        }
    ],
}

print(f'User: {initial_msg}')
updated_messages = agent.invoke(messages)
messages = updated_messages
print(f'Agent: {messages["messages"][-1].content}')
print(f'Updated Messages:')
display(messages['messages'])

User: I need to procure products for 30 blt sandwiches. Create a valid product order for me.
Agent: It appears there is no bacon product available in the current database. Would you like me to proceed with ordering the bread, lettuce, and tomato for the BLT sandwiches, or would you like to search for a substitute or alternative supplier for bacon?
Updated Messages:


[HumanMessage(content='I need to procure products for 30 blt sandwiches. Create a valid product order for me.', additional_kwargs={}, response_metadata={}, id='d08abb8f-9c0b-41e1-a89e-72a56c2c658f'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 314, 'total_tokens': 514, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D6uXJGowcJlogULZRMlUtnEYmEpNh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3c63-4a9a-7430-bc32-4686b7a6a511-0', tool_calls=[{'name': 'product_search', 'args': {'sql_query': "SELECT * FROM products WHERE name LIKE '%bacon%' OR name LIKE '%lettuce%' OR name 

In [14]:
# Continue the conversation with follow-up messages
if msg := input('Enter follow-up user message (or press Enter to skip): '):
    messages['messages'].append({
        'role': 'user',
        'content': msg
    })
    print(f'User: {msg}')
    updated_messages = agent.invoke(messages)
    print(f'Agent: {updated_messages["messages"][-1].content}')
    print(f'Updated Messages:')
    display(updated_messages['messages'])

User: yes
Agent: There is no lettuce product available in the current database either. I will proceed with ordering the bread and tomato for the BLT sandwiches. Since bacon and lettuce are not available, you may need to source them separately. Here is the adjusted product order for 30 BLT sandwiches with the available items:

- Whole Wheat Bread Loaf: 30 loaves (assuming each loaf has enough slices for 1 sandwich)
- Canned Diced Tomatoes: 30 cans (as a tomato substitute)

Would you like me to create this order for you?
Updated Messages:


[HumanMessage(content='I need to procure products for 30 blt sandwiches. Create a valid product order for me.', additional_kwargs={}, response_metadata={}, id='d08abb8f-9c0b-41e1-a89e-72a56c2c658f'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 314, 'total_tokens': 514, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D6uXJGowcJlogULZRMlUtnEYmEpNh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3c63-4a9a-7430-bc32-4686b7a6a511-0', tool_calls=[{'name': 'product_search', 'args': {'sql_query': "SELECT * FROM products WHERE name LIKE '%bacon%' OR name LIKE '%lettuce%' OR name 